In [4]:
from deck import full_euchre_deck
from dealer import Dealer
from sim_game import game_sim
from sim_game_list_comp import lc_game_sim
from branch_calc import calc_all_possible_hands, filter_branch_by_hand
import numpy as np

In [5]:
game = Dealer(deck=full_euchre_deck ,players=4)
game.deal_cards()
hands5 = [game.hand1, game.hand2, game.hand3, game.hand4]
v_hands = 5*[hands5]
hands5
# first_play = [game.hand1[0], game.hand1[1], game.hand1[2], game.hand1[3], game.hand1[4]]

[array([[ 10,   0],
        [ 11,   0],
        [-12,   0],
        [  0, 140],
        [  0, -14]]),
 array([[  0, 120],
        [  0, 135],
        [  0, -12],
        [  0, -10],
        [ 14,   0]]),
 array([[-13,   0],
        [-14,   0],
        [  0,  -9],
        [  9,   0],
        [  0,  90]]),
 array([[  0, 130],
        [ 13,   0],
        [  0, 100],
        [-10,   0],
        [-11,   0]])]

In [6]:
game_sim(dealt_hands=hands5)

12
129
1109
5314


0.8242378622506586